### Explainer Experiment Demo

Below we show some examples on how you can use the **Experiment_Explainers** class. The general steps consist of:
1. prepare data
2. Initialize recommenders, explainers and metrics
3. Initialize experiment and run the experiment

You can set distribution=True to enable distribution plots while running the experiments. The plots will be saved in /experiment_plots.

When the experiment is completed, result table is saved in /experiment_plots.

In [1]:
import warnings
warnings.filterwarnings("ignore")

**Experiment 1**: run PSPNFNS, FDIV and FPR on (FM, Exp_LIMERS) pair using rating data and sentiment data from amazon toys. Since this dataset does not contain explicit item and user features, we parsed aspect and opinion from sentiment data as item and user features respectively in the  ***create_item_features_from_aspects*** function.

In [5]:
from cornac.datasets import amazon_toy
import numpy as np
from cornac.data import FeatureModality
from cornac.eval_methods import RatioSplit
from cornac.experiment import Experiment_Explainers
from cornac.models import FMRec
from cornac.explainer import Exp_LIMERS
from cornac.metrics_explainer import (
    Metric_Exp_PSPNFNS as PSPN,
    Metric_Exp_DIV as DIV,
    Metric_Exp_FPR as FPR,
)


def create_item_features_from_aspects(at_sentiment):
    """Separate aspects and opinions from sentiment data and create item and user features from them."""
    items = {}
    users = {}
    for _, row in enumerate(at_sentiment):
        user, item, sentiments = row
        if user not in users:
            users[user] = []
        if item not in items:
            items[item] = []
        for sentiment in sentiments:
            if sentiment[0] not in items[item]:
                items[item].append(sentiment[0])  # aspect adds to item feature
            if sentiment[1] not in users[user]:
                users[user].append(sentiment[1])  # opinion adds to user feature

    item_aspect_pairs = np.array(
        [(item, feature) for item in items for feature in items[item]]
    )
    user_opinion_pairs = np.array(
        [(user, feature) for user in users for feature in users[user]]
    )
    return item_aspect_pairs, user_opinion_pairs, items.keys(), users.keys()


at_feedback = amazon_toy.load_feedback()
at_feedback = at_feedback[: len(at_feedback) // 20]  # reduce data size
at_sentiment = amazon_toy.load_sentiment()
items_feature, users_feature, items_list, users_list = (
    create_item_features_from_aspects(at_sentiment)
)
# remove unknown users and items from rating data
at_feedback_excl_unknowns = [
    x for x in at_feedback if x[0] in users_list and x[1] in items_list
]

In [6]:
# prepare data
rs = RatioSplit(
    data=at_feedback,
    test_size=0.2,
    item_feature=FeatureModality(items_feature),
    # user_feature=FeatureModality(users_feature), # user feature is not used in this experiment
    seed=42,
    exclude_unknowns=True,
)
# initialize recommenders, explainers and metrics
fm = FMRec()
limers = Exp_LIMERS(rec_model=fm, dataset=rs.train_set)
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()

# initialize experiment
models = [(fm, limers)]
metrics = [pspnfns, fdiv, fpr]
experiment = Experiment_Explainers(
    eval_method=rs,
    models=models,
    metrics=metrics,
    distribution=True,
    rec_k=4,
    feature_k=4,
    eval_train=True,
)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender FMRec...


Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.84335
-- Epoch 2
Training MSE: 0.54054
-- Epoch 3
Training MSE: 0.47963
-- Epoch 4
Training MSE: 0.43793
-- Epoch 5
Training MSE: 0.40798
-- Epoch 6
Training MSE: 0.38155
-- Epoch 7
Training MSE: 0.36740
-- Epoch 8
Training MSE: 0.35302
-- Epoch 9
Training MSE: 0.34088
-- Epoch 10


INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'FMRec:Exp_LIMERS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender FMRec creates recommendations


Training MSE: 0.33139


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_LIMERS create explanation for all recommendations
Computing explanations: : 20537it [07:39, 44.66it/s]                         
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PSPNFNS starts evaluation...
INFO:cornac.experiment.experiment_explainers:self.current_rec: FMRec, self.current_exp: Exp_LIMERS
Re-evaluate after features removal... : 20537it [06:49, 50.21it/s]                           
INFO:cornac.experiment.experiment_explainers:Result: Probability of Necessity: 0.9873071979434447; Probability of Sufficiency: 0.0; Harmonic Mean: 0.0
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.08820979829202373
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_FPR d

**Experiment 2**: run PSPN, FDIV and FPR on (EFM, Exp_EFM), (MTER, Exp_MTER), (ComparERObj, Exp_ComparERObj) and (ComparERSub, Exp_ComparERSub)

In [1]:
from cornac.experiment import Experiment_Explainers
from cornac.models import MTER, EFM, ComparERObj, ComparERSub
from cornac.explainer import Exp_EFM, Exp_MTER, Exp_ComparERObj, Exp_ComparERSub, Exp_Counter
from cornac.metrics_explainer import (
    Metric_Exp_PSPNFNS as PSPN,
    Metric_Exp_DIV as DIV,
    Metric_Exp_FPR as FPR,
)
from cornac.datasets import amazon_toy
from cornac.data.reader import Reader
from cornac.eval_methods import StratifiedSplit
from cornac.data.sentiment import SentimentModality

rating = amazon_toy.load_feedback(fmt="UIRT", reader=Reader(min_user_freq=50))
sentiment_data = amazon_toy.load_sentiment(reader=Reader(min_user_freq=50))

md = SentimentModality(data=sentiment_data)

eval_method = StratifiedSplit(
    data=rating,
    group_by="user",
    chrono=True,
    sentiment=md,
    test_size=0.2,
    exclude_unknowns=True,
    verbose=True,
)

# initialize recommenders and explainers
efm = EFM(
    max_iter=20,
    num_explicit_factors=128,
    num_latent_factors=128,
    num_most_cared_aspects=100,
    rating_scale=5.0,
    alpha=0.9,
    lambda_x=1,
    lambda_y=1,
    lambda_u=0.01,
    lambda_h=0.01,
    lambda_v=0.01,
    trainable=True,
)
efm_exp = Exp_EFM(rec_model=efm, dataset=eval_method.train_set)
counter = Exp_Counter(rec_model=efm, dataset=eval_method.train_set, rec_k=10)
mter = MTER(
    max_iter=20,
    n_user_factors=8,
    n_item_factors=8,
    n_aspect_factors=8,
    n_opinion_factors=8,
    n_bpr_samples=1000,
    n_element_samples=50,
    lambda_reg=0.1,
    lambda_bpr=10,
    lr=0.5,
)
mter_exp = Exp_MTER(rec_model=mter, dataset=eval_method.train_set)

efm.fit(eval_method.train_set)
params = {
        "U1": efm.U1,
        "U2": efm.U2,
        "H1": efm.H1,
        "H2": efm.H2,
        "V": efm.V,
}
comparerobj = ComparERObj(
    max_iter=20,
    num_explicit_factors=128,
    num_latent_factors=128,
    num_most_cared_aspects=20,
    rating_scale=5.0,
    alpha=0.7,
    lambda_x=1,
    lambda_y=1,
    lambda_u=0.01,
    lambda_h=0.01,
    lambda_v=0.01,
    lambda_d=0.1,
    min_user_freq=2,
    trainable=True,
    verbose=True,
    init_params=params,
)
exp_comparerobj = Exp_ComparERObj(comparerobj, eval_method.train_set)

mter.fit(eval_method.train_set)
params = {
    "G1": mter.G1,
    "G2": mter.G2,
    "G3": mter.G3,
    "U": mter.U,
    "I": mter.I,
    "A": mter.A,
    "O": mter.O,
}

comparersub = ComparERSub(
    max_iter=20,
    n_user_factors=8,
    n_item_factors=8,
    n_aspect_factors=8,
    n_opinion_factors=8,
    n_pair_samples=1000,
    n_bpr_samples=1000,
    n_element_samples=50,
    lambda_reg=0.1,
    lambda_bpr=10,
    lambda_d=10,
    lr=0.5,
    min_common_freq=1,
    min_user_freq=2,
    min_pair_freq=1,
    trainable=True,
    verbose=True,
    init_params=params,
)
exp_comparersub = Exp_ComparERSub(comparersub, eval_method.train_set)


# initialize metrics
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()
fpr_with_input_as_groundtruth = FPR(ground_truth=sentiment_data)

# initialize experiment
models = [(efm, efm_exp), (efm, counter), (mter, mter_exp), (comparerobj, exp_comparerobj), (comparersub, exp_comparersub)]
metrics = [fdiv, fpr_with_input_as_groundtruth, pspnfns]
experiment = Experiment_Explainers(
    eval_method=eval_method,
    models=models,
    metrics=metrics,
    rec_k=10,
    distribution=False,
    feature_k=10,
    eval_train=True,
)
experiment.run()

/Users/yaoyao/miniconda3/envs/rose/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yaoyao/miniconda3/envs/rose/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 119
Number of items = 4058
Number of ratings = 7197
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 119
Number of items = 4058
Number of ratings = 741
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 119
Total items = 4058


INFO:cornac.experiment.experiment_explainers:Start training Recommender EFM...
/Users/yaoyao/miniconda3/envs/rose/lib/python3.9/site-packages/cornac-2.0.0-py3.9-macosx-11.1-arm64.egg/cornac/models/recommender.py:307: UserWarning: Model is already fitted. Re-fitting will overwrite the previous model.
  warnings.warn(
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender EFM creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 1190/1190 [00:00<00:00, 5580.88it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.6092330960971876
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
Start

**Experiment 3**: run FDIV, PGF, MEP and EnDCG on (ALS, Exp_ALS), (EMF, Exp_PHI4MF) and (NEMF, Exp_SU4EMF) pairs using data from movielens. This dataset only contains user, item and rating info.

In [1]:
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.data.reader import Reader
from cornac.experiment.experiment_explainers import Experiment_Explainers
from cornac.models import EMF, NEMF, ALS
from cornac.explainer import Exp_ALS, Exp_PHI4MF, Exp_SU4EMF
from cornac.metrics_explainer import (
    Metric_Exp_DIV as DIV,
    Metric_Exp_PGF as PGF,
    Metric_Exp_MEP as MEP,
    Metric_Exp_EnDCG as EnDCG,
)

# Load MovieLens
data = movielens.load_feedback(variant="100K", reader=Reader(min_user_freq=150))

# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=data, test_size=0.2, exclude_unknowns=False, verbose=True
)

# initialize recommenders and explainers
emf = EMF(
    k=10,
    max_iter=500,
    learning_rate=0.001,
    lambda_reg=0.1,
    explain_reg=0.01,
    verbose=True,
    seed=6,
    num_threads=6,
    early_stop=True,
)
nemf = NEMF(
    k=10,
    max_iter=500,
    learning_rate=0.001,
    lambda_reg=0.1,
    explain_reg=0.01,
    novel_reg=1,
    verbose=True,
    seed=6,
    num_threads=6,
    early_stop=True,
)
als = ALS(k=10, max_iter=500, lambda_reg=0.001, alpha=1, verbose=True, seed=6)
als_exp = Exp_ALS(rec_model=als, dataset=ratio_split.train_set)
emf_exp = Exp_PHI4MF(rec_model=emf, dataset=ratio_split.train_set)
nemf_exp = Exp_SU4EMF(rec_model=nemf, dataset=ratio_split.train_set)

# initialize metrics
fdiv = DIV()
pgf = PGF()
mep = MEP()
endcg = EnDCG()

# initialize experiment
models = [(emf, emf_exp), (als, als_exp), (nemf, nemf_exp)]
metrics = [fdiv, pgf, mep, endcg]
experiment = Experiment_Explainers(
    eval_method=ratio_split,
    models=models,
    metrics=metrics,
    distribution=False,
    rec_k=10,
    feature_k=10,
    eval_train=True,
)
experiment.run()

/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
INFO:cornac.experiment.experiment_explainers:Start training Recommender EMF...


rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 230
Number of items = 1619
Number of ratings = 46456
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 230
Number of items = 1653
Number of ratings = 11615
Number of unknown users = 0
Number of unknown items = 34
---
Total users = 230
Total items = 1653


  0%|          | 0/500 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EMF:Exp_PHI4MF'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender EMF creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_PHI4MF create explanation for all recommendations


Optimization finished!


...

  0%|          | 0/500 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'NEMF:Exp_SU4EMF'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender NEMF creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_SU4EMF create explanation for all recommendations


Optimization finished!


Computing explanations:   0%|          | 0/2300 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.2297882783392449
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 230/230 [00:00<00:00, 319.37it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.02494628222598492
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_MEP starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_MEP: 0.29130434782608694
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_EnDCG starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_EnDCG: 0.12516964557088964
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.003886325252432064, 0.08064328547486145, 0.6843478260869565, 0.3148327388807963, 2.9156980514526367, 231.64698433876038], [0.0361

**Experiment 4**: Run RA and FA on pairwise models. FA and RA performs comparison between two sets of explainers, thus the score is unavailable when only one explainer is passed.

In [5]:
from cornac.experiment import Experiment_Explainers
from cornac.models import MTER, EFM, ComparERObj, ComparERSub, TriRank
from cornac.explainer import (
    Exp_EFM,
    Exp_MTER,
    Exp_ComparERObj,
    Exp_ComparERSub,
    Exp_TriRank,
)
from cornac.datasets import amazon_toy
from cornac.data.reader import Reader
from cornac.eval_methods import StratifiedSplit
from cornac.data.sentiment import SentimentModality
from cornac.metrics_explainer import (
    Metric_Exp_FA as FA,
    Metric_Exp_RA as RA,
)


rating = amazon_toy.load_feedback(fmt="UIRT", reader=Reader(min_user_freq=50))
sentiment_data = amazon_toy.load_sentiment(reader=Reader(min_user_freq=50))

md = SentimentModality(data=sentiment_data)

eval_method = StratifiedSplit(
    data=rating,
    group_by="user",
    chrono=True,
    sentiment=md,
    test_size=0.2,
    exclude_unknowns=True,
    verbose=True,
)

# initialize recommenders and explainers
efm = EFM(
    max_iter=20,
    num_explicit_factors=128,
    num_latent_factors=128,
    num_most_cared_aspects=100,
    rating_scale=5.0,
    alpha=0.9,
    lambda_x=1,
    lambda_y=1,
    lambda_u=0.01,
    lambda_h=0.01,
    lambda_v=0.01,
    trainable=True,
)
efm_exp = Exp_EFM(rec_model=efm, dataset=eval_method.train_set)

mter = MTER(
    max_iter=20,
    n_user_factors=8,
    n_item_factors=8,
    n_aspect_factors=8,
    n_opinion_factors=8,
    n_bpr_samples=1000,
    n_element_samples=50,
    lambda_reg=0.1,
    lambda_bpr=10,
    lr=0.5,
)
mter_exp = Exp_MTER(rec_model=mter, dataset=eval_method.train_set)

efm.fit(eval_method.train_set)
params = {
        "U1": efm.U1,
        "U2": efm.U2,
        "H1": efm.H1,
        "H2": efm.H2,
        "V": efm.V,
}
comparerobj = ComparERObj(
    max_iter=20,
    num_explicit_factors=128,
    num_latent_factors=128,
    num_most_cared_aspects=20,
    rating_scale=5.0,
    alpha=0.7,
    lambda_x=1,
    lambda_y=1,
    lambda_u=0.01,
    lambda_h=0.01,
    lambda_v=0.01,
    lambda_d=0.1,
    min_user_freq=2,
    trainable=True,
    verbose=True,
    init_params=params,
)
exp_comparerobj = Exp_ComparERObj(comparerobj, eval_method.train_set)

mter.fit(eval_method.train_set)
params = {
    "G1": mter.G1,
    "G2": mter.G2,
    "G3": mter.G3,
    "U": mter.U,
    "I": mter.I,
    "A": mter.A,
    "O": mter.O,
}

comparersub = ComparERSub(
    max_iter=20,
    n_user_factors=8,
    n_item_factors=8,
    n_aspect_factors=8,
    n_opinion_factors=8,
    n_pair_samples=1000,
    n_bpr_samples=1000,
    n_element_samples=50,
    lambda_reg=0.1,
    lambda_bpr=10,
    lambda_d=10,
    lr=0.5,
    min_common_freq=1,
    min_user_freq=2,
    min_pair_freq=1,
    trainable=True,
    verbose=True,
    init_params=params,
)
exp_comparersub = Exp_ComparERSub(comparersub, eval_method.train_set)

trirank = TriRank(
    verbose=True,
    seed=123,
)
exp_trirank = Exp_TriRank(rec_model=trirank, dataset=eval_method.train_set)
# initialize metrics
fa = FA()
ra = RA()

# initialize experiment
models = [
    [(efm, mter), (efm_exp, mter_exp)],
    [(efm, comparerobj), (efm_exp, exp_comparerobj)],
    [(mter, comparersub), (mter_exp, exp_comparersub)],
    [(efm, trirank), (efm_exp, exp_trirank)]
]
metrics = [fa, ra]
experiment = Experiment_Explainers(
    eval_method=eval_method,
    models=models,
    metrics=metrics,
    rec_k=10,
    feature_k=10,
    eval_train=True,
    distribution=False,
)
experiment.run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 119
Number of items = 4058
Number of ratings = 7197
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 119
Number of items = 4058
Number of ratings = 741
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 119
Total items = 4058


INFO:cornac.experiment.experiment_explainers:Start training Recommender1 EFM...
INFO:cornac.experiment.experiment_explainers:Start training Recommender2 MTER...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'vs'MTER:Exp_MTER'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Creates fake recommendations from dataset for common used
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer1 Exp_EFM create explanation for all recommendations


Computing explanations:   0%|          | 0/119 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer2 Exp_MTER create explanation for all recommendations


Computing explanations:   0%|          | 0/119 [00:00<?, ?it/s]

...

Building matrices completed in 0 s


Computing explanations:   0%|          | 0/119 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer2 Exp_TriRank create explanation for all recommendations


Computing explanations:   0%|          | 0/119 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FA starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Average Metric_Exp_FA: 0.1478991596638654
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_RA starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Average Metric_Exp_RA: 0.007563025210084033
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.1176470588235293, 0.06576630652260904, 0.9829080104827881, 0.4178743362426758], [1.0, 1.0, 128.8874969482422, 0.10132598876953125], [0.9943977591036414, 0.992530345471522, 2113.6905925273895, 0.611400842666626], [0.1478991596638654, 0.007563025210084033, 1.3195741176605225, 0.22526264190673828]]
INFO:cornac.experiment.experiment_explainers:Experiment result: 
 recommender:explainer                        |      Metric_Exp_FA |        Metric_Exp_RA |           Train(s) |         Evaluate(s)
EFM:Exp_EFM'vs'MTER:Exp_MTER                 | 0.1176

**Experiment 5**: Run FDIV with (NARRE, Exp_NARRE) with amazon_digital_music

In [2]:
from cornac.datasets import amazon_digital_music
from cornac.eval_methods import RatioSplit
from cornac.data.reader import Reader
from cornac.data import ReviewModality
from cornac.data.text import BaseTokenizer
from cornac.experiment import Experiment_Explainers
from cornac.models import NARRE
from cornac.explainer import Exp_NARRE
from cornac.metrics_explainer import Metric_Exp_DIV


feedback = amazon_digital_music.load_feedback(reader=Reader(min_user_freq=50))
reviews = amazon_digital_music.load_review(reader=Reader(min_user_freq=50))


review_modality = ReviewModality(
    data=reviews,
    tokenizer=BaseTokenizer(stop_words="english"),
    max_vocab=4000,
    max_doc_freq=0.5,
)

ratio_split = RatioSplit(
    data=feedback,
    test_size=0.1,
    val_size=0.1,
    exclude_unknowns=True,
    review_text=review_modality,
    verbose=True,
    seed=123,
)

pretrained_word_embeddings = {}  # You can load pretrained word embedding here

narre = NARRE(
    embedding_size=100,
    id_embedding_size=32,
    n_factors=32,
    attention_size=16,
    kernel_sizes=[3],
    n_filters=64,
    dropout_rate=0.5,
    max_text_length=50,
    batch_size=64,
    max_iter=10,
    init_params={'pretrained_word_embeddings': pretrained_word_embeddings},
    verbose=True,
    seed=123,
)

narre_exp = Exp_NARRE(rec_model=narre, dataset=ratio_split.train_set)

div = Metric_Exp_DIV()

experiment = Experiment_Explainers(
    eval_method=ratio_split,
    models=[(narre, narre_exp)],
    metrics=[div],
    rec_k=10,
    feature_k=10,
    eval_train=True,
    distribution=False,
)

experiment.run()

/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 126
Number of items = 2829
Number of ratings = 9809
Max rating = 5.0
Min rating = 1.0
Global mean = 4.2
---
Test data:
Number of users = 126
Number of items = 2829
Number of ratings = 1129
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 126
Number of items = 2829
Number of ratings = 1121
---
Total users = 126
Total items = 2829


INFO:cornac.experiment.experiment_explainers:Start training Recommender NARRE...


Number of OOV words: 4004


  0%|          | 0/10 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'NARRE:Exp_NARRE'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender NARRE creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_NARRE create explanation for all recommendations


Learning completed!


Computing explanations:   0%|          | 0/1260 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.3986967484915915
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.3986967484915915, 540.1019442081451, 2.2283430099487305]]
INFO:cornac.experiment.experiment_explainers:Experiment result: 
 recommender:explainer |     Metric_Exp_DIV |          Train(s) |        Evaluate(s)
NARRE:Exp_NARRE       | 0.3986967484915915 | 540.1019442081451 | 2.2283430099487305



**Experiment 6**: Run both experiment and experiment_explainers 

- Experiment: NDCG, AUC withe TriRank
- Experiment_Explainers: Metric_Exp_DIV with (TriRank, Exp_TriRank) 
- Dataset: amazon_toy


In [1]:
from cornac.datasets import amazon_toy
from cornac.data import SentimentModality
from cornac.eval_methods import RatioSplit
from cornac.data.reader import Reader
from cornac.experiment import Experiment_Explainers, Experiment
from cornac.models import TriRank
from cornac.explainer import Exp_TriRank
from cornac.metrics_explainer import Metric_Exp_DIV
from cornac.metrics import NDCG, AUC

# Load rating and sentiment information
rating = amazon_toy.load_feedback(reader=Reader(min_user_freq=50))
sentiment = amazon_toy.load_sentiment(reader=Reader(min_user_freq=50))


# Instantiate a SentimentModality, it makes it convenient to work with sentiment information
md = SentimentModality(data=sentiment)


# Define an evaluation method to split feedback into train and test sets
eval_method = RatioSplit(
    data=rating,
    test_size=0.15,
    exclude_unknowns=True,
    verbose=True,
    sentiment=md,
    seed=123,
)

# Instantiate the model
trirank = TriRank(
    verbose=True,
    seed=123,
)
# Instantiate the explainer
exp_trirank = Exp_TriRank(rec_model=trirank, dataset=eval_method.train_set)

# Instantiate evaluation metrics
ndcg_50 = NDCG(k=50)
auc = AUC()

# Run the experiment for the TriRank model
Experiment(
    eval_method=eval_method, models=[trirank], metrics=[ndcg_50, auc]
).run()

# initialize metrics for explainers
div = Metric_Exp_DIV()

# Run the experiment for the TriRank model with the explainer
Experiment_Explainers(
    eval_method=eval_method,
    models=[(trirank, exp_trirank)],
    metrics=[div],
    rec_k=10,
    feature_k=10,
    eval_train=True,
    distribution=False,
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 119
Number of items = 4235
Number of ratings = 7698
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 119
Number of items = 4235
Number of ratings = 836
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 119
Total items = 4235

[TriRank] Training started!
Building matrices started!


Building matrices:   0%|          | 0/114 [00:00<?, ?it/s]

Building symmetric normalized matrices R, X, Y
Building matrices completed in 0 s

[TriRank] Evaluation started!


Ranking:   0%|          | 0/117 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Start training Recommender TriRank...


...

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'TriRank:Exp_TriRank'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender TriRank creates recommendations


Building matrices completed in 0 s


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_TriRank create explanation for all recommendations


Computing explanations:   0%|          | 0/1190 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.24104642698256118
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.24104642698256118, 0.8859922885894775, 8.317068815231323]]
INFO:cornac.experiment.experiment_explainers:Experiment result: 
 recommender:explainer |      Metric_Exp_DIV |           Train(s) |       Evaluate(s)
TriRank:Exp_TriRank   | 0.24104642698256118 | 0.8859922885894775 | 8.317068815231323

